# **Proyecto 1 - Inteligencia Artificial**

Adriana Palacios 23044
Bryan Martínez 23542

---

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer

from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from xgboost.callback import EarlyStopping

import pickle

## **Exploración y Comprensión de los Datos**

In [ ]:
# features and target variables
features_path = "datasets/training_features.csv"
target_path = "datasets/training_target.csv"

features = pd.read_csv(features_path)
target = pd.read_csv(target_path).iloc[:, 0]

### **Inspección de Dimensiones del Dataset**

In [ ]:
features.shape
target.shape

print("Dimensiones features:", features.shape)
print("Dimensiones target:", target.shape)

El dataset cuenta con 2344 observaciones y 80 variables.

### **Tipos de Variables, Identificación y Valores Faltantes**


| Variable        | Tipo       | Subtipo  | ¿Qué es?                                                        | ¿Hay valores faltantes? |
| --------------- | ---------- | -------- | --------------------------------------------------------------- | ---------------------- |
| MS SubClass     | Categórica | Nominal  | Los números representan el tipo de vivienda.                    | No                     |
| MS Zoning       | Categórica | Nominal  | Identifica el tipo de zona.                                     | No                     |
| Lot Frontage    | Numérica   | Continua | Mide cuánto mide el frente del terreno del lado de la calle.    | Sí                     |
| Lot Area        | Numérica   | Continua | Tamaño del lote.                                                | No                     |
| Street          | Categórica | Nominal  | Tipo de carretera que da a la propiedad.                        | No                     |
| Alley           | Categórica | Nominal  | Tipo de callejón que da a la propiedad.                         | Sí                     |
| Lot Shape       | Categórica | Ordinal  | Forma de la propiedad.                                          | No                     |
| Land Contour    | Categórica | Nominal  | Qué tan plana es la propiedad.                                  | No                     |
| Utilities       | Categórica | Nominal  | Tipos de utilidades disponibles en la propiedad.                | No                     |
| Lot Config      | Categórica | Nominal  | Exposición del terreno a calles.                                | No                     |
| Land Slope      | Categórica | Ordinal  | Inclinación de la propiedad                                     | No                     |
| Neighborhod     | Categórica | Nominal  | Vecindario en la que está.                                      | No                     |
| Condition 1     | Categórica | Nominal  | Proximidad a algunas condiciones.                               | No                     |
| Condition 2     | Categórica | Nominal  | Proximidad a condiciones (más de 1 está presente).              | No                     |
| Bldg Type       | Categórica | Nominal  | Tipo de vivienda.                                               | No                     |
| House Style     | Categórica | Nominal  | Estilo de la vivienda (niveles, estado de estos).               | No                     |
| Overall Qual    | Categórica | Ordinal  | Calificación del material y acabados de la casa.                | No                     |
| Overall Cond    | Categórica | Ordinal  | Calificación de la condición de la casa.                        | No                     |
| Year Built      | Numérica   | Discreta | Fecha de construcción original                                  | No                     |
| Year Remod/Add  | Numérica   | Discreta | Fecha de remodelación, si no hubo, es la original               | No                     |
| Roof Style      | Categórica | Nominal  | Tipo de techo                                                   | No                     |
| Roof Matl       | Categórica | Nominal  | Material del techo.                                             | No                     |
| Exterior 1st    | Categórica | Nominal  | Cubierta exterior de la casa.                                   | No                     |
| Exterior 2nd    | Categórica | Nominal  | Cubierta exterior de la casa, si es que hay más de un material. | No                     |
| Mas Vnr Type    | Categórica | Nominal  | Tipo de revestimiento de mampostería.                           | Sí                     |
| Mas Vnr Area    | Numérica   | Continua | Área de mampostería.                                            | Sí                     |
| Exter Qual      | Categórica | Ordinal  | Calidad del material en el exterior.                            | No                     |
| Exter Cond      | Categórica | Ordinal  | Evalúa la condición del material en el exterior.                | No                     |
| Foundation      | Categórica | Nominal  | Tipo de base de la casa.                                        | No                     |
| Bsmt Qual       | Categórica | Ordinal  | Evalúa la altura del sótano.                                    | Sí                     |
| Bmst Cond       | Categórica | Ordinal  | Evalúa la calidad general del sótano.                           | Sí                     |
| Bmst Exposure   | Categórica | Ordinal  | Exposición del sótano.                                          | Sí                     |
| BmstFin Type 1  | Categórica | Nominal  | Calificación del área principal.                                | Sí                     |
| BsmtFin SF 1    | Numérica   | Continua | Tamaño de zona 1.                                               | Sí                     |
| BsmtFin Type 2  | Categórica | Nominal  | Calificación de segunda área del sótano.                        | Sí                     |
| BsmtFin SF 2    | Numérica   | Continua | Pies cuadrados terminados de segunda área.                      | Sí                     |
| Bsmt Unf SF     | Numérica   | Continua | Medida de área sin terminar del sótano.                         | Sí                     |
| Total Bsmt SF   | Numérica   | Continua | Área total del sótano.                                          | Sí                     |
| Heating         | Categórica | Nominal  | Tipo de calentamiento en la vivienda.                           | No                     |
| HetingQC        | Categórica | Ordinal  | Calidad y condición del calientamiento.                         | No                     |
| Central Air     | Categórica | Nominal  | Si tiene aire acondicionado.                                    | No                     |
| Electrical      | Categórica | Nominal  | Sistema eléctrico                                               | No                     |
| 1sr Flr SF      | Numérica   | Continua | Área del primer nivel.                                          | No                     |
| 2nd Flr SF      | Numérica   | Continua | Área del segundo nivel.                                         | No                     |
| Low Qual Fin SF | Numérica   | Continua | Área de baja calidad en todos los niveles.                      | No                     |
| Gr Liv Area     | Numérica   | Continua | Área arriba de la tierra.                                       | No                     |
| Bsmt Full Bath  | Numérica   | Discreta | Cantidad de baños completos en el sotáno.                       | Sí                     |
| Bsmt Half Bath  | Numérica   | Discreta | Cantidad de baños a la mitad.                                   | Sí                     |
| Full Bath       | Numérica   | Discreta | Cantidad de baños completos en la casa.                         | No                     |
| Half Bath       | Numérica   | Discreta | Cantidad de baños a la mitad.                                   | No                     |
| Bedroom AbvGr   | Númerica   | Discreta | Dormitorios dentro de la vivienda.                              | No                     |
| Kitchen AbvGr   | Numérica   | Discreta | Cantidad de cocinas dentro de la vivienda.                      | No                     |
| Kitchen Qual    | Categórica | Ordinal  | Calidad de la cocina.                                           | No                     |
| TotRmsAbvGrd    | Numérica   | Discreta | Cantidad de cuartos dentro de la vivienda.                      | No                     |
| Functional      | Categórica | Ordinal  | Nivel de funcionalidad de la casa.                              | No                     |
| Fireplaces      | Numérica   | Discreta | Número de chimeneas.                                            | No                     |
| Fireplace Qu    | Categórica | Ordinal  | Calidad de la chimenea.                                         | Sí                     |
| Garage Type     | Categórica | Nominal  | Tipos de garage.                                                | Sí                     |
| Garage Yr Blt   | Numérica   | Discreta | Año en que se construyó el garage.                              | Sí                     |
| Garage Finish   | Categórica | Ordinal  | Cómo se encuentra el interior del garage.                       | Sí                     |
| Garage Cars     | Numérica   | Discreta | Capacidad de carros.                                            | Sí                     |
| Garage Area     | Numérica   | Continua | Área del garage.                                                | Sí                     |
| Garage Qual     | Categórica | Ordinal  | Calidad del garage.                                             | Sí                     |
| Garage Cond     | Categórica | Ordinal  | Condición en la que se encuentra el garage.                     | Sí                     |
| Paved Drive     | Categórica | Ordinal  | Qué tan pavimentada está la entrada.                            | No                     |
| Wood Deck SF    | Numérica   | Continua | Área de cubierta de madera.                                     | No                     |
| Open Porch SF   | Numérica   | Continua | Área de porche abierto.                                         | No                     |
| Enclosed Porch  | Numérica   | Continua | Área de porche cerrado.                                         | No                     |
| 3-Sssn Porch    | Numérica   | Continua | Área de un porche de tres temporadas.                           | No                     |
| Screen Porch    | Numérica   | Continua | Área con porche de mosquitero.                                  | No                     |
| Pool Area       | Numérica   | Continua | Área de la piscina.                                             | No                     |
| Pool QC         | Categórica | Ordinal  | Calidad de la piscina.                                          | Sí                     |
| Fence           | Categórica | Ordinal  | Calidad de la cerca.                                            | Sí                     |
| Misc Feature    | Categórica | Nominal  | Indica si la propiedad tiene alguna característica adicional.   | Sí                     |
| Misc Val        | Numérica   | Continua | Valor de la característica adicional.                           | Sí                     |
| Mo Sold         | Numérica   | Discreta | Mes en que se vendió.                                           | No                     |
| Yr Sold         | Numérica   | Discreta | Año en que se vendió.                                           | No                     |
| Sale Type       | Categórica | Nominal  | Tipo de acuerdo en el que se vendió la casa.                    | No                     |
| Sale Condition  | Categórica | Nominal  | Condición de la venta.                                          | No                     |
| Sale Price      | Numérica   | Continua | Precio de venta.                                                | No                     |


### **Análisis Preliminar de Distribuciones y Posibles Outliers**

In [ ]:
categoricas_con_numeros = [
    "MS SubClass",
    "Overall Qual",
    "Overall Cond",
]

numeric_cols = features.select_dtypes(include=[np.number]).columns
numeric_cols_reales = [c for c in numeric_cols if c not in categoricas_con_numeros]

for col in numeric_cols_reales:
    plt.figure()
    plt.boxplot(features[col].dropna().values, showfliers=True)
    plt.title(col)
    plt.ylabel(col)
    plt.show()


**Análisis de Outliers**

- Lot Frontage: presenta varios puntos atípicos, tenemos un dato con un área frente a la casa por encima de los 300 pies cuadrados, y varios que se posicionan por encima de de 130 pies cuadrados. Cabe mencionar que incluso hay outliers que están en cero.
- Lot Area: hay una alta presencia de valores atípicos para viviendas que tienen un tamaño de terreno muy por encima de los demás.
- Year Built: hay una ligera presencia de outliers, pero estos los encontramos para casas casas construidas previo a 1900.
- Year Remod/Add: no hay valores atípicos para esta variable.
- Mas Vnr Area: parece que hay pocas casas que cuentan con bastante área de mampostería, hay varios puntos atípicos que se encuentran por encima de los 400 pies cuadrados.
- BsmtFin SF 1: hay una leve presencia de valores atípicos para viviendas con un área principal de sótano considerablemente grande, con valores por encima de los 2000 pies cuadrados.
- BsmtFin SF 2: se evidencia que es peculiar que las viviendas cuenten con un área secundaria de sótano, analizando el diagrama de caja y bigote, parece que el hecho de contar con un valor distinto a cero ya se considera como algo muy diferente a la mayoría de las viviendas.
- Bsmt Unf SF: los outliers se encuentran en valores altos de área sin terminar para el sótano, estando por encima de 1600 pies cuadrados.
- Total Bsmt SF: para esta variable se considera como puntos atípicos, el hecho de no contar con sotáno, o tener un área arriba de los 2000 pies cuadrados.
- 1st Flr SF: hay varios puntos atípicos, todos estos se encuentran para áreas de primer nivel por encima de los 2000 pies cuadrados.
- 2nd Flr SF: se tiene baja presencia de outliers, pero estos los podemos identificar en casas que tienen más de 1700 pies cuadrados.
- Low Qual Fin SF: aquí se evidencia que el hecho de contar área de baja calidad en la vivienda es considerado como algo atípico entre las viviendas.
- Gr Liv Area: los outliers los identificamos en viviendas con un alto valor de área arriba de la tierra, estos se encuentran por encima de los 2600 pies cuadrados.
- Bsmt Full Bath: en este caso solamente tenemos un punto atípico, y se da con una vivienda que cuenta con tres baños completos en el sótano.
- Bsmt Half Bath: hay poca presencia de valores atípicos, la mayoría de viviendas tienen 0 baños a la mitad, y todas aquellas que tienen uno o dos baños de este estilo son outliers.
- Full Bath: para ser considerado como punto atípico se tiene que contar con 4 baños completos dentro de la casa.
- Half Bath: no hay presencia de valores atípicos, todas las viviendas están entre 0 y 2.
- Bedroom AbvGr: hay algunos puntos atípicos, estos los podemos encontrar en viviendas con 0 dormitorios dentro de la vivienda, o en aquellas que tienen más de cuatro.
- Kitchen AbvGr: los outliers los identificamos en viviendas que no tienen cocina, o que cuentan con más de una.
- TotRms AbvGr: los valores atípicos ocurrene n las viviendas que tienen más de 10 cuartos en la vivienda.
- Fireplaces: en cuanto a chimeneas, es anormal ver viviendas con más de 2.
- Garage Yr Blt: aquí notamos algo muy peculiar, ya que parece ser que existe una observación con un valor arriba del año 2200, esto es algo imposible, por lo que se tendrá que limpiar esto del dataset previo a hacer el split. Los demas outliers se encuentran antes de 1900.
- Garage Cars: las viviendas que tienen espacio para más de tres carros en el garage son valores atípicos para la variable.
- Garage Area: los valores atípicos se encuentran en áreas grandes de garage, en aquellas que tienen más de 900 pies cuadrados.
- Wood Deck SF: los outliers se encuentran en viviendas con área de cubierta de madera arriba de 400 pies cuadrados.
- Open Porch SF: tenemos varios valores atípicos, los podemos encontrar en viviendas con un porche abierto que tengan un área por encima de 150 pies cuadrados.
- Enclosed Porch: en este caso, los valores atípicos los vemos evidenciados en viviendas que tienen un porche cerrado, es decir, valor de área distinto de cero.
- 3Ssn Proch: se considera como outlier a los valores distintos de cero, en otras palabras, el hecho de contar con un porche de tres temporadas es algo inusual.
- Screen Porch: vamos que el hecho de contar con un porche de mosquitero es considerado como algo atípico.
- Pool Area: las viviendas con piscina son puntos atípicos, por lo que vemos que el hecho de tener un área distinta de cero es algo inusual.
- Misc Val: se presentan valores atípicos en aquellos puntos que cuentan con un valor para una característica adicional, es decir, ser mayor a 0 para esta variable.
- Mo Sold: no hay valores atípicos.
- Yr Sold: no se tienen valores atípicos.

### **Identificación de Posibles Correlaciones Relevantes**

In [ ]:
numeric_features = features.select_dtypes(include=[np.number])

corr_target = numeric_features.corrwith(target).sort_values(key=lambda s: s.abs(), ascending=False)

pd.set_option("display.max_rows", None)
corr_df = corr_target.to_frame("corr_with_target")
corr_df